In [83]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (154 kB/s)
Reading package lists... Done


In [84]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-31 18:26:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.58MB/s    in 0.2s    

2020-10-31 18:26:10 (5.58 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [85]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [86]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [87]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [88]:
# Step 1: Create a new DF: condition => total_votes>=20
filtered_vine_df = vine_df.filter('total_votes>=20')
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [89]:
# Step 2: Create a new DF: condition => helpful_votes/total_votes >= 50%
helpful_votes50_df = filtered_vine_df.withColumn('grater50', filtered_vine_df['helpful_votes'] / filtered_vine_df['total_votes']).filter('grater50>=0.5')
helpful_votes50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|0.8620689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|               1.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|0.9354838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|0.9629629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|0.9523809523809523|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|0.8974358974358975|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|               1.0|
|R1E1PQGHWGO3Q1|          1|  

In [90]:
# Step 3: Create a new DF: condition => vine=='Y'
vine_Y_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'Y')
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|               1.0|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|0.9285714285714286|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|0.9634146341463414|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|               0.6|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|0.8648648648648649|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|0.8484848484848485|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|0.9076923076923077|
|R1W3KCDHM6BMZT|          4|  

In [91]:
# Step 4: Create a new DF: condition => vine=='N'
vine_N_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'N')
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|0.8620689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|               1.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|0.9354838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|0.9629629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|0.9523809523809523|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|0.8974358974358975|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|               1.0|
|R1E1PQGHWGO3Q1|          1|  

In [92]:
# Step 5.1: vine_reviews_YES: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews'
# Step 5.1.a: total number of reviews
vineY_reviews_TOTAL_df = vine_Y_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Vine_Reviews")
vineY_reviews_TOTAL_df.show()

+------------------+
|Total_Vine_Reviews|
+------------------+
|               107|
+------------------+



In [93]:
# Step 5.1.b: 5-star revievs 
vineY_reviews_5STAR_df = vine_Y_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_Vine_Reviews")
vineY_reviews_5STAR_df.show()

+--------------------+
|5-stars_Vine_Reviews|
+--------------------+
|                  56|
+--------------------+



In [95]:
vineY_reviews_percentage = vineY_reviews_5STAR_df.collect()[0]['5-stars_Vine_Reviews'] / vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews'] * 100
round(vineY_reviews_percentage,2)

52.34

In [94]:
# Step 5.2: vine_reviews_NO: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews '
# Step 5.2.a: total number of reviews
vineN_reviews_TOTAL_df = vine_N_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_non-Vine_Reviews")
vineN_reviews_TOTAL_df.show()

+----------------------+
|Total_non-Vine_Reviews|
+----------------------+
|                 39869|
+----------------------+



In [96]:
# Step 5.2.a: number of 5-stars reviews
vineN_reviews_5STAR_df = vine_N_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_non-Vine_Reviews")
vineN_reviews_5STAR_df.show()

+------------------------+
|5-stars_non-Vine_Reviews|
+------------------------+
|                   21005|
+------------------------+



In [97]:
vineN_reviews_percentage = vineN_reviews_5STAR_df.collect()[0]['5-stars_non-Vine_Reviews'] / vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews'] * 100
round(vineN_reviews_percentage,2)

52.69

In [98]:
# Create data frame for % results
percentage_df = spark.createDataFrame([
                                       (52.34, 52.69)],
                                      ['Percentage Vine Reviews', "Percentage non-Vine Reviews"])
percentage_df.show()

+-----------------------+---------------------------+
|Percentage Vine Reviews|Percentage non-Vine Reviews|
+-----------------------+---------------------------+
|                  52.34|                      52.69|
+-----------------------+---------------------------+



In [99]:
# Positivity Bias - additional calculations
# Vine Reviews
all_stars_YES = vine_Y_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews Vine")


In [101]:
# Vine Reviews (cont)
import pyspark.sql.functions as func
all_stars_YES_percentage = all_stars_YES.withColumn('% of total', func.round(all_stars_YES['Count of Reviews Vine']/ \
                                                (vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews']/100), 2)) \
                                                .orderBy(all_stars_YES['star_rating'].asc()).show()

+-----------+---------------------+----------+
|star_rating|Count of Reviews Vine|% of total|
+-----------+---------------------+----------+
|          1|                    1|      0.93|
|          2|                    2|      1.87|
|          3|                   16|     14.95|
|          4|                   32|     29.91|
|          5|                   56|     52.34|
+-----------+---------------------+----------+



In [102]:
# non-Vine Reviews
all_stars_NO = vine_N_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews non-Vine")

In [103]:
# non-Vine Reviews (cont)
all_stars_NO_percentage = all_stars_NO.withColumn('% of total', func.round(all_stars_NO['Count of Reviews non-Vine']/ \
                                            (vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews']/100), 2))  \
                                            .orderBy(all_stars_NO['star_rating'].asc()).show()

+-----------+-------------------------+----------+
|star_rating|Count of Reviews non-Vine|% of total|
+-----------+-------------------------+----------+
|          1|                     5324|     13.35|
|          2|                     2412|      6.05|
|          3|                     3744|      9.39|
|          4|                     7384|     18.52|
|          5|                    21005|     52.69|
+-----------+-------------------------+----------+

